In [3]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 mins 56 secs
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,2 months and 29 days
H2O_cluster_name:,H2O_from_python_davidnunez_uudt4f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.957 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [34]:
model_undersampling = h2o.load_model('/Users/davidnunez/Desktop/tesis/tesis-model/Training/train2021/randomOverSamplingModel/StackedEnsemble_AllModels_1_AutoML_1_20230726_231013')
model_oversampling = h2o.load_model('/Users/davidnunez/Desktop/tesis/tesis-model/Training/train2021/smoteModel/StackedEnsemble_AllModels_1_AutoML_3_20230726_232848')
model_unbalanced = h2o.load_model('/Users/davidnunez/Desktop/tesis/tesis-model/Training/train2021/unbalancedModel/StackedEnsemble_AllModels_1_AutoML_4_20230726_232950')

In [35]:
y = 'Protestas'
df_2022 = pd.read_csv('../df_dummizado_from_2021.csv', index_col=False)
df_2022 = df_2022[df_2022['date.year'] == 2022]
test = h2o.H2OFrame.from_python(df_2022)
test[y] = test[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [36]:
preds_2022_under = model_undersampling.predict(test)
preds_2022_over = model_oversampling.predict(test)
preds_2022_unb = model_unbalanced.predict(test)


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [39]:
total_cmp = df_2022[df_2022['Protestas'] == True]

In [29]:
total_cmp.shape

(3318, 932)

## Undersampling test

In [40]:
df_preds_2022_under = preds_2022_under.as_data_frame()
df_preds_2022_under['result'] = df_preds_2022_under['True'] >= 0.5
df_preds_2022_under['is_right'] = df_preds_2022_under['result'] == df_preds_2022_under['predict']
# df_preds.head()
# GROUP BY 
gkk = df_preds_2022_under.groupby(['is_right'], dropna=False)
gkk.count()
# df_preds_2022_under.head()

,predict,False,True,result
is_right,,,,
False,2245,2245,2245,2245
True,19739,19739,19739,19739


## Oversampling test

In [45]:
df_preds_2022_over = preds_2022_over.as_data_frame()
df_preds_2022_over['result'] = df_preds_2022_over['True'] >= 0.5
df_preds_2022_over['is_right'] = df_preds_2022_over['result'] == df_preds_2022_over['predict']
gkk = df_preds_2022_over.groupby(['is_right'], dropna=False)
gkk.count()

,predict,False,True,result
is_right,,,,
False,42,42,42,42
True,21942,21942,21942,21942


In [46]:
(3318 - 42)/3318

0.9873417721518988

## Unbalanced test

In [41]:
df_preds_2022_unb = preds_2022_unb.as_data_frame()
df_preds_2022_unb['result'] = df_preds_2022_unb['True'] >= 0.5
df_preds_2022_unb['is_right'] = df_preds_2022_unb['result'] == df_preds_2022_unb['predict']
gkk = df_preds_2022_unb.groupby(['is_right'], dropna=False)
gkk.count()

,predict,False,True,result
is_right,,,,
False,950,950,950,950
True,21034,21034,21034,21034


In [44]:
(3318 - 950)/3318

0.7136829415310428